### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar
data = pd.read_json("./arxivData.json")
data.sample(n=5)

x arxivData.json


,author,day,id,link,month,summary,tag,title,year
26751,"[{'name': 'Florian Walch'}, {'name': 'Caner Ha...",23,1611.07890v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,In this work we propose a new CNN+LSTM archite...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Image-based localization using LSTMs for struc...,2016
31768,"[{'name': 'Maxim Raginsky'}, {'name': 'Rebecca...",15,0911.2904v4,"[{'rel': 'related', 'href': 'http://dx.doi.org...",11,This paper describes a methodology for detecti...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Sequential anomaly detection in the presence o...,2009
14180,"[{'name': 'Patryk Chrabaszcz'}, {'name': 'Ilya...",24,1802.08842v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Evolution Strategies (ES) have recently been d...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Back to Basics: Benchmarking Canonical Evoluti...,2018
28665,"[{'name': 'Jayakorn Vongkulbhisal'}, {'name': ...",13,1707.04318v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Many computer vision problems are formulated a...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Discriminative Optimization: Theory and Applic...,2017
25276,"[{'name': 'Limin Wang'}, {'name': 'Yuanjun Xio...",8,1507.02159v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Deep convolutional networks have achieved grea...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Towards Good Practices for Very Deep Two-Strea...,2015


In [5]:
# assemble lines: concatenate title and description
lines = data.apply(
    lambda row: row["title"] + " ; " + row["summary"].replace("\n", " "), axis=1
).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [10]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
for i, line in enumerate(lines):
    lines[i] = " ".join(tokenizer.tokenize(line)).lower()

In [11]:
assert (
    sorted(lines, key=len)[0]
    == "differential contrastive divergence ; this paper has been retracted ."
)
assert (
    sorted(lines, key=len)[2]
    == "p = np ; we claim to resolve the p =? np problem via a formal argument for p = np ."
)

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [37]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        line = " ".join([UNK] * (n - 1)) + " " + line + " " + EOS
        l = 0
        r = n
        while r <= len(line.split()):
            counts[tuple([token for token in line.split()[l : r - 1]])][
                line.split()[r - 1]
            ] += 1
            l += 1
            r += 1

    return counts

In [34]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[("_UNK_", "_UNK_")]) == 78
assert dummy_counts["_UNK_", "a"]["note"] == 3
assert dummy_counts["p", "="]["np"] == 2
assert dummy_counts["author", "."]["_EOS_"] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [38]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix, token_counter in counts.items():
            count_sum = sum(token_counter.values())
            for token, token_count in token_counter.items():
                self.probs[prefix][token] = token_count / count_sum

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1) :]
        prefix = [UNK] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [39]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens("")  # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial["learning"], 0.02)
assert np.allclose(p_initial["a"], 0.13)
assert np.allclose(p_initial.get("meow", 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens("a")  # '' -> ['_UNK_', 'a']
assert np.allclose(p_a["machine"], 0.15384615)
assert np.allclose(p_a["note"], 0.23076923)
assert np.allclose(p_a.get("the", 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens("a note")["on"], 1)
assert dummy_lm.get_possible_next_tokens(
    "a machine"
) == dummy_lm.get_possible_next_tokens(
    "there have always been ghosts in a machine"
), "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [40]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [116]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    old = lm.get_possible_next_tokens(prefix)
    new_probs = np.zeros(len(old.values()))
    for i, prob in enumerate(old.values()):
        new_probs[i] = prob ** (1 / (temperature + 1e-2))

    new_probs /= sum(new_probs)
    next_token = np.random.choice(list(old.keys()), p=new_probs, size=1)
    return next_token[0]

In [117]:
from collections import Counter

test_freqs = Counter([get_next_token(lm, "there have") for _ in range(10000)])
assert 250 < test_freqs["not"] < 450
assert 8500 < test_freqs["been"] < 9500
assert 1 < test_freqs["lately"] < 200

test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=1.0) for _ in range(10000)]
)
assert 1500 < test_freqs["learning"] < 3000
test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=0.5) for _ in range(10000)]
)
assert 8000 < test_freqs["learning"] < 9000
test_freqs = Counter(
    [get_next_token(lm, "deep", temperature=0.0) for _ in range(10000)]
)
assert test_freqs["learning"] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [118]:
prefix = "artificial"  # <- your ideas :)

for i in range(100):
    prefix += " " + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial neural network for sar image segmentation ; in particular , we find experimentally that the proposed methodology than learning from crowds ; unmanned aircraft system ( extended version of label queries to service difference ). _EOS_


In [119]:
prefix = "bridging the"  # <- more of your ideas

for i in range(100):
    prefix += " " + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the input data . we present a novel approach based on the performance of the world , and ( b ) a novel multi - armed bandit problem with an additional challenge is how to employ the softmax non - linear functions . the proposed approach is based on the other hand , the proposed method is based on the use of a set of operations o ( n log n )$ for rna - seq data . we demonstrate the advantages of the human visual system ; in this paper is to find a low - rank


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [123]:
lm.get_next_token_prob("there", "is")

0.3333333333333333

In [197]:
def perplexity(lm, lines, min_logprob=np.log(10**-50.0)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    perp = 0
    N = 0
    for line in lines:
        N += len(line.split())
        prefix = ""
        for token in line.split():
            prob = np.log(lm.get_next_token_prob(prefix, token))
            if np.isinf(prob):
                prob = min_logprob
            perp += prob
            prefix += " " + token
    perp = np.exp(perp / (-N))

    return perp

In [195]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(
    lm3, ["the jabberwock , with eyes of flame , "]
)  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(
    0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)
), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10**6, (
    "missing words should have large but finite perplexity. "
    " Make sure you use min_logprob right"
)

Perplexities: ppx1=346.980 ppx3=1.539 ppx10=1.190


C:\Users\user\AppData\Local\Temp\ipykernel_4364\3840160236.py:17: RuntimeWarning: divide by zero encountered in log
  prob = np.log(lm.get_next_token_prob(prefix, token))


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split

train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    """this code is an example, no need to change anything"""

    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(
            token for token_counts in counts.values() for token in token_counts
        )
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {
                token: (token_counts[token] + delta) / total_count
                for token in token_counts
            }

    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total)  # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [ ]:
# test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(
        sum([dummy_lm.get_next_token_prob("a", w_i) for w_i in dummy_lm.vocab]), 1
    ), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

In [ ]:
# optional: try to sample tokens from such a model